In [1]:
import tensorflow as tf

## numpy와 tensor

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html


- 개념적으로는 같지만 tensor는 deep-learning 관점에서 더 편하게 쓸 수 있음
- 기능은 numpy가 더 많음.
- functional paradigm 관점에서는 mutable한거 안 좋음.(tensor는 ummutable)
- layer를 numpy보다 tensor로 구현하는게 더 깔끔함.
- tensor는 gpu지원
- tensor는 자동 미분 가능

**control flow**
    - if, for 등으로 동적으로 사용 가능
    - declarative 방법은 control flow 지원X(Sequential)

In [2]:
# tensor
a = tf.constant([1,2,3])   # 소문자
b = tf.Variable([1,2,3])   # 대문자 -> class

In [5]:
b.assign([4,5,6])
b
# 값을 바꿀 수 있다, 개수는 같아야
# a는 안됨 상수라서

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([4, 5, 6])>

In [6]:
for i in dir(b):
    if 'assign' in i:
        print(i)

_assign_dependencies
_strided_slice_assign
assign
assign_add
assign_sub


In [8]:
tf.add([1,2],[3,4]) # element wise 형태로 받음

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 6])>

In [9]:
a.numpy()   # bound나오면 함수처럼 실행

array([1, 2, 3])

In [11]:
a = tf.constant([1,2,3])
b = tf.constant([4,5,6])
import numpy as np
np.add(a,b)

array([5, 7, 9])

In [12]:
c = np.array([1,2,3])
d = np.array([4,5,6])
tf.add(c,d)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([5, 7, 9])>

**hdf5** : 대용량 데이터를 저장하기 위한 format

In [13]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [16]:
input_ = tf.keras.Input(shape=(28,28))
flatten = tf.keras.layers.Flatten()(input_)  # __call__이 있어서 가능
dense1 = tf.keras.layers.Dense(128,activation = 'relu')(flatten)
dense2 = tf.keras.layers.Dense(10,activation = 'softmax')(dense1)

# input을 flatten안에 집어넣기 -> 합성함수처럼 사용

### Model
- **functional paradigm** 활용
- model summary에서
    - input에서 리스트 형태  -> multi input, output 가능
- 다양하고 복잡한 모델

In [18]:
model = tf.keras.models.Model(inputs=input_, outputs=dense2)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [29]:
vgg = tf.keras.applications.MobileNetV2()
# vgg.summary()

In [23]:
issubclass(vgg.__class__,tf.keras.models.Model)

True

In [25]:
# tf.keras.utils.plot_model(model,show_shapes=True)

In [30]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2546 - accuracy: 0.9276
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1128 - accuracy: 0.9668
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0780 - accuracy: 0.9775
Epoch 4/5
1875/1875 [==============================] - 2s 969us/step - loss: 0.0582 - accuracy: 0.9821
Epoch 5/5
1875/1875 [==============================] - 2s 952us/step - loss: 0.0437 - accuracy: 0.9868


In [26]:
issubclass(type(tf.keras.models.Sequential),type(tf.keras.models.Model))

True

In [27]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])
# dir(model2)  __call__ 있다  -> 합성함수처럼 사용 가능

In [7]:
input_ = tf.keras.Input(shape=(28,28))
x = model2(input_)

In [8]:
model3 = tf.keras.models.Model(inputs=input_, outputs=x)
model3.summary()

# 모델안에 모델

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 10)                101770    
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [34]:
model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model3.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2973 - accuracy: 0.9140
Epoch 2/5
1875/1875 [==============================] - 2s 996us/step - loss: 0.1466 - accuracy: 0.9563
Epoch 3/5
1875/1875 [==============================] - 2s 967us/step - loss: 0.1084 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0889 - accuracy: 0.9723
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0761 - accuracy: 0.9757


In [36]:
model2(x_train)

<tf.Tensor: shape=(60000, 10), dtype=float32, numpy=
array([[1.1948140e-13, 4.6771989e-11, 6.1089848e-11, ..., 1.1043370e-10,
        1.3663265e-11, 1.7848075e-08],
       [9.9999416e-01, 2.4756577e-10, 4.4985550e-06, ..., 4.6018584e-08,
        9.0223229e-10, 2.9223159e-07],
       [4.6845543e-08, 3.4678371e-05, 7.9638907e-05, ..., 1.1373491e-03,
        4.5500467e-05, 3.6220276e-03],
       ...,
       [9.6277656e-12, 6.4861325e-11, 5.2026263e-15, ..., 4.0459307e-12,
        8.3487933e-08, 3.5144842e-07],
       [4.9452814e-05, 1.4055942e-07, 2.5144587e-05, ..., 1.3239364e-07,
        4.4089651e-07, 9.5063641e-08],
       [4.6108627e-05, 6.9188829e-08, 2.5990910e-06, ..., 4.2898014e-06,
        9.9947459e-01, 2.1395573e-04]], dtype=float32)>

한 장만 넣기

In [37]:
model2(x_train[0][tf.newaxis])  # 새로운 차원

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[1.1948094e-13, 4.6771989e-11, 6.1089730e-11, 5.9163786e-04,
        9.6198582e-19, 9.9940836e-01, 2.1268093e-15, 1.1043349e-10,
        1.3663239e-11, 1.7848041e-08]], dtype=float32)>

In [45]:
tf.expand_dims(x_train[0],0).shape  # 0으로 새로운 차원

TensorShape([1, 28, 28])

In [47]:
print(tf.expand_dims(x_train[0],2).shape)
print(x_train[0][:,:,tf.newaxis].shape)
print(x_train[0][...,tf.newaxis].shape)
# 같은 거임

(28, 28, 1)
(28, 28, 1)
(28, 28, 1)


In [41]:
model2(tf.expand_dims(x_train[0],0))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[1.1948094e-13, 4.6771989e-11, 6.1089730e-11, 5.9163786e-04,
        9.6198582e-19, 9.9940836e-01, 2.1268093e-15, 1.1043349e-10,
        1.3663239e-11, 1.7848041e-08]], dtype=float32)>

In [42]:
model2.predict(tf.expand_dims(x_train[0],0))

array([[1.1948094e-13, 4.6771989e-11, 6.1089730e-11, 5.9163786e-04,
        9.6198582e-19, 9.9940836e-01, 2.1268093e-15, 1.1043349e-10,
        1.3663239e-11, 1.7848041e-08]], dtype=float32)


같은 predict가 다르게 행동 -> 객체 지향에서 다형성

In [62]:
print(x_train[0][None].shape)
print(x_train[0][np.newaxis].shape)

(1, 28, 28)
(1, 28, 28)


In [57]:
model2(x_train[0][None]).numpy()

array([[1.1948094e-13, 4.6771989e-11, 6.1089730e-11, 5.9163786e-04,
        9.6198582e-19, 9.9940836e-01, 2.1268093e-15, 1.1043349e-10,
        1.3663239e-11, 1.7848041e-08]], dtype=float32)

In [56]:
model2(x_train[0][None]).numpy().sum()

1.0

In [51]:
model2.predict(x_train[0][None])

array([[1.1948094e-13, 4.6771989e-11, 6.1089730e-11, 5.9163786e-04,
        9.6198582e-19, 9.9940836e-01, 2.1268093e-15, 1.1043349e-10,
        1.3663239e-11, 1.7848041e-08]], dtype=float32)

In [61]:
model2(x_train[0][None]) is model2.predict(x_train[0][None])
# 메모리가 다르기 때문에

False

In [54]:
np.array_equal(model2(x_train[0][None]), model2.predict(x_train[0][None]))

True

In [58]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


#### model summary
- None : 몇 개가 들어올지 모르지만 자리만 잡아주고 넘기는 것

### input 없이 하기

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# summary를 보면 input이 없어서 에러 난다.
# 근데 데이터 넣어서 나오고 학습 후 summary도 나옴

In [4]:
model2.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [5]:
model2(x_train[0][tf.newaxis])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.06573551, 0.21200478, 0.0752316 , 0.04274188, 0.12666884,
        0.12735763, 0.1253439 , 0.06726936, 0.07872646, 0.07891998]],
      dtype=float32)>

In [5]:
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model2.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.3006 - accuracy: 0.9127
Epoch 2/5
1875/1875 [==============================] - 2s 980us/step - loss: 0.1433 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 2s 996us/step - loss: 0.1073 - accuracy: 0.9670
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0866 - accuracy: 0.9735
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0751 - accuracy: 0.9765


In [6]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


모델에 값이 들어갈 때 그거에 따라 결정해줌

None이 안나오면 의심해봐야 함.

In [4]:
x = np.arange(9).reshape(1,3,3)
model2(x)
model2.summary()
# 모양을 결정시켜버림 -> x_train으로 fit하면 에러

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1280      
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 2,570
Trainable params: 2,570
Non-trainable params: 0
_________________________________________________________________


In [5]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [6]:
model2.layers

In [15]:
model3.layers[1].layers